## Week 1
### Data Capture from Binance


In [1]:
pip install websockets pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import asyncio
import websockets
import json
import pandas as pd
import datetime
import os

In [4]:
buffer = []

async def binanceDepthCollector():
    dep_url = f"wss://stream.binance.com:9443/ws/btcusdt@depth10@100ms"
    async with websockets.connect(dep_url) as websocket:
        print('Connected')
        while True:
            try:
                info = await websocket.recv()
                #print('start listenning') #测一下
                data = json.loads(info)
                now = datetime.datetime.now()
                timestamp = now.strftime('%Y-%m-%d %H:%M:%S.%f')

                rows = []#存储当前访问的depth

                #print(data)#测一下读取到的数据格式

                for i, (bid,ask) in enumerate(zip(data['bids'],data['asks'])):
                    price1, qty1 = bid
                    price2, qty2 = ask
                    rows.append([timestamp,data['lastUpdateId'],[float(price1),float(qty1)],[float(price2),float(qty2)],i+1])

                buffer.extend(rows)#准备该次访问写入的数据
                
                df = pd.DataFrame(buffer, columns = ['timestamp','lastUpdateId','bids','asks','level'])
                filename = "depth10_btcusdt_live.csv"
                header_adj = not os.path.exists(filename)
                df.to_csv(filename, mode = 'a', header = header_adj, index = False)
                #print('file saved')

                buffer.clear()#清空缓存

            except Exception as e:
                print(f"[Error] {e}")
                await asyncio.sleep(5)  

In [5]:
buffer.clear()#清空缓存
await binanceDepthCollector()

Connected


CancelledError: 

In [ ]:
await websocket.close()